### prophet

In [2]:
import pandas as pd
from prophet import Prophet

In [3]:
train_data = pd.read_csv('snp_price_phophet_train.csv')
test_data = pd.read_csv('snp_price_phophet_test.csv')

In [4]:
train_data

,ds,stock,y
0,2018-01-03 00:00:00,A,1.025444
1,2018-01-04 00:00:00,A,1.017751
2,2018-01-05 00:00:00,A,1.034024
3,2018-01-08 00:00:00,A,1.036243
4,2018-01-09 00:00:00,A,1.061686
...,...,...,...
547259,2022-04-22 00:00:00,ZTS,2.604193
547260,2022-04-25 00:00:00,ZTS,2.580577
547261,2022-04-26 00:00:00,ZTS,2.532916
547262,2022-04-27 00:00:00,ZTS,2.554385


In [5]:
test_data

,ds,stock,y
0,2022-04-29 00:00:00,A,1.823123
1,2022-05-02 00:00:00,A,1.827709
2,2022-05-03 00:00:00,A,1.870967
3,2022-05-04 00:00:00,A,1.930581
4,2022-05-05 00:00:00,A,1.867146
...,...,...,...
136811,2023-05-23 00:00:00,ZTS,2.492338
136812,2023-05-24 00:00:00,ZTS,2.441386
136813,2023-05-25 00:00:00,ZTS,2.385093
136814,2023-05-26 00:00:00,ZTS,2.377876


In [6]:
prophet_models = {}
for stock in train_data['stock'].unique():
    # Filter data for the specific stock
    df_stock = train_data[train_data['stock'] == stock]

    # Create a new Prophet model for the stock
    model = Prophet()

    # Fit the model to the stock's data
    model.fit(df_stock)

    # Store the fitted model in the dictionary
    prophet_models[stock] = model

11:55:57 - cmdstanpy - INFO - Chain [1] start processing
11:55:58 - cmdstanpy - INFO - Chain [1] done processing
11:55:58 - cmdstanpy - INFO - Chain [1] start processing
11:55:58 - cmdstanpy - INFO - Chain [1] done processing
11:55:58 - cmdstanpy - INFO - Chain [1] start processing
11:55:59 - cmdstanpy - INFO - Chain [1] done processing
11:55:59 - cmdstanpy - INFO - Chain [1] start processing
11:56:00 - cmdstanpy - INFO - Chain [1] done processing
11:56:00 - cmdstanpy - INFO - Chain [1] start processing
11:56:00 - cmdstanpy - INFO - Chain [1] done processing
11:56:00 - cmdstanpy - INFO - Chain [1] start processing
11:56:01 - cmdstanpy - INFO - Chain [1] done processing
11:56:01 - cmdstanpy - INFO - Chain [1] start processing
11:56:02 - cmdstanpy - INFO - Chain [1] done processing
11:56:02 - cmdstanpy - INFO - Chain [1] start processing
11:56:02 - cmdstanpy - INFO - Chain [1] done processing
11:56:03 - cmdstanpy - INFO - Chain [1] start processing
11:56:03 - cmdstanpy - INFO - Chain [1]

In [7]:
stock_list =[]
for x in test_data['stock'].unique():
    stock_list.append(x)

prediction dataframe 

In [8]:
prophet_pred = pd.DataFrame(columns=['ds'] + stock_list)  # Initialize the forecasted values dataframe

for stock in stock_list:
    df_stock_test = test_data[test_data['stock'] == stock]
    model = prophet_models[stock]
    future = model.make_future_dataframe(periods=len(df_stock_test))
    forecast = model.predict(future)
    predicted_values = forecast[['ds', 'yhat']].tail(len(df_stock_test))
    
    # Rename the 'yhat' column to the stock symbol
    predicted_values = predicted_values.rename(columns={'yhat': stock})
    
    prophet_pred = pd.merge(prophet_pred, predicted_values, on='ds', how='outer')  # Merge the predicted values with the main dataframe

prophet_pred['ds'] = pd.to_datetime(prophet_pred['ds'])  # Convert 'ds' column to datetime if not already

# Sort the dataframe based on the 'ds' column
prophet_pred = prophet_pred.sort_values('ds').reset_index(drop=True)

# Print the forecasted values dataframe
print(prophet_pred)


     A_x AAL_x AAP_x AAPL_x ABBV_x ABC_x ABT_x ACGL_x ACN_x ADBE_x  ...  \
0    NaN   NaN   NaN    NaN    NaN   NaN   NaN    NaN   NaN    NaN  ...   
1    NaN   NaN   NaN    NaN    NaN   NaN   NaN    NaN   NaN    NaN  ...   
2    NaN   NaN   NaN    NaN    NaN   NaN   NaN    NaN   NaN    NaN  ...   
3    NaN   NaN   NaN    NaN    NaN   NaN   NaN    NaN   NaN    NaN  ...   
4    NaN   NaN   NaN    NaN    NaN   NaN   NaN    NaN   NaN    NaN  ...   
..   ...   ...   ...    ...    ...   ...   ...    ...   ...    ...  ...   
267  NaN   NaN   NaN    NaN    NaN   NaN   NaN    NaN   NaN    NaN  ...   
268  NaN   NaN   NaN    NaN    NaN   NaN   NaN    NaN   NaN    NaN  ...   
269  NaN   NaN   NaN    NaN    NaN   NaN   NaN    NaN   NaN    NaN  ...   
270  NaN   NaN   NaN    NaN    NaN   NaN   NaN    NaN   NaN    NaN  ...   
271  NaN   NaN   NaN    NaN    NaN   NaN   NaN    NaN   NaN    NaN  ...   

       WYNN_y     XEL_y     XOM_y    XRAY_y     XYL_y     YUM_y     ZBH_y  \
0    0.519746  1.70834

In [9]:
prophet_pred = prophet_pred.loc[:, ~prophet_pred.columns.str.endswith('_x')]

# Rename remaining columns to remove '_y' suffix and keep only stock names
prophet_pred.columns = prophet_pred.columns.str.replace('_y', '')

# Print the modified dataframe
print(prophet_pred)

            ds         A       AAL       AAP      AAPL      ABBV       ABC  \
0   2022-04-29  1.977556  0.301744  2.176896  4.137167  2.006712  1.806538   
1   2022-04-30  1.951938  0.261104  2.165170  4.212221  1.926960  1.736530   
2   2022-05-01  1.951174  0.259805  2.166557  4.214890  1.932725  1.737538   
3   2022-05-02  1.971126  0.299528  2.180363  4.144673  2.022752  1.807990   
4   2022-05-03  1.969958  0.299132  2.186592  4.153666  2.027265  1.809611   
..         ...       ...       ...       ...       ...       ...       ...   
267 2023-01-21  1.928109  0.176870  2.517608  5.196478  2.429723  2.089061   
268 2023-01-22  1.924286  0.176866  2.513704  5.199494  2.432911  2.092544   
269 2023-01-23  1.941241  0.217953  2.522129  5.130015  2.520183  2.165527   
270 2023-01-24  1.937172  0.219007  2.522925  5.140128  2.521804  2.169727   
271 2023-01-25  1.933241  0.218276  2.515912  5.145230  2.525497  2.172903   

          ABT      ACGL       ACN  ...      WYNN       XEL     

In [10]:
prophet_pred.set_index('ds', inplace=True)  # Set 'ds' column as the index
prophet_pred.index.name = 'Date'  # Rename the index to 'Date'

# Print the modified dataframe
print(prophet_pred)

                   A       AAL       AAP      AAPL      ABBV       ABC  \
Date                                                                     
2022-04-29  1.977556  0.301744  2.176896  4.137167  2.006712  1.806538   
2022-04-30  1.951938  0.261104  2.165170  4.212221  1.926960  1.736530   
2022-05-01  1.951174  0.259805  2.166557  4.214890  1.932725  1.737538   
2022-05-02  1.971126  0.299528  2.180363  4.144673  2.022752  1.807990   
2022-05-03  1.969958  0.299132  2.186592  4.153666  2.027265  1.809611   
...              ...       ...       ...       ...       ...       ...   
2023-01-21  1.928109  0.176870  2.517608  5.196478  2.429723  2.089061   
2023-01-22  1.924286  0.176866  2.513704  5.199494  2.432911  2.092544   
2023-01-23  1.941241  0.217953  2.522129  5.130015  2.520183  2.165527   
2023-01-24  1.937172  0.219007  2.522925  5.140128  2.521804  2.169727   
2023-01-25  1.933241  0.218276  2.515912  5.145230  2.525497  2.172903   

                 ABT      ACGL       

now I keep it save for furthur use

In [ ]:
prophet_pred.to_csv('price_prophet_pred.csv',index=True)